In [1]:
from flask import Blueprint
import numpy as np
import pandas as pd
import pickle
import copy
from sklearn.preprocessing import MinMaxScaler
import requests
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
from pandas import Series
from numpy import nan
import copy
import math

In [55]:
from glob import glob
import os
import pandas as pd
# from flask import jsonify/

path = os.getcwd()
csv_list = sorted(glob(os.path.join(path,"DB","*.csv")))

data = pd.read_csv(csv_list[1])
data.iloc[0,0]

5.1

In [300]:
from flask import Blueprint
from glob import glob
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def train(model, x, y):
  with tf.GradientTape() as t:
    # Trainable variables are automatically tracked by GradientTape
    current_loss = loss(y_true=y,y_pred=model(x, training=True))

  # Use GradientTape to calculate the gradients with respect to W and b
  grads= t.gradient(current_loss, model.trainable_variables)
  
  # Subtract the gradient scaled by the learning rate
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

def training_loop(model, x, y, validset = None):
  for epoch in epochs:
    # Update the model with the single giant batch
    train(model, x, y)

    # Track this before I update
    if validset == None:
        current_loss = loss(y,model(x))
        print("Epoch %2d: loss=%2.5f" %
            (epoch, current_loss))
    else:
        x_val = validset[0]
        y_val = validset[1]

        current_loss = loss(y_true=y,y_pred=model(x))
        valid_loss = loss(y_true=y_val,y_pred=model(x_val))
        print("Epoch %2d: loss=%2.5f  val_loss=%2.5f" %
            (epoch, current_loss, valid_loss))

#데이터 셋
path = os.getcwd()
csv_list = sorted(glob(os.path.join(path,"DB","*.csv")))
data = pd.read_csv(csv_list[1])
trainset, testset = train_test_split(data,test_size=0.2,shuffle=True)

scaler = MinMaxScaler()

X_train = trainset.values[:,:-1]
X_train = scaler.fit_transform(X_train)
y_train = trainset.values[:,-1]
# y_train = pd.get_dummies(trainset.values[:,-1]).values
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)


X_test = testset.values[:,:-1]
X_test = scaler.transform(X_test)
y_test = testset.values[:,-1]
# y_test = pd.get_dummies(testset.values[:,-1]).values
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


#params
input_shape = 4
output_shape = 3
lr = 0.03
act_function = 'relu'
outlayer_act_function = 'softmax'
category = "classification"
#optimizer
optimizer = keras.optimizers.SGD(learning_rate=lr)

# #loss function
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


num_perceptron_layer1 = 8
num_perceptron_layer2 = 16
num_perceptron_layer3 = 32
num_perceptron_layer4 = 16
num_perceptron_layer5 = 8
num_perceptron_out = output_shape

num_perceptron_list = [
    num_perceptron_layer1,
    num_perceptron_layer2,
    num_perceptron_layer3,
    num_perceptron_layer4,
    num_perceptron_layer5,
    num_perceptron_out,
    ]

# model definition
inputs = tf.keras.Input(shape=input_shape)
for i in range(len(num_perceptron_list)):
    n_percep = num_perceptron_list[i]
    if i == 0:
        x = keras.layers.Dense(n_percep, activation= act_function)(inputs)

    elif  i != 0 and n_percep != 0 and i != len(num_perceptron_list)-1:
        x = keras.layers.Dense(n_percep, activation= act_function)(x)

    elif  i == len(num_perceptron_list)-1:
        x = keras.layers.Dense(n_percep,)(x)#activation= outlayer_act_function)(x)

    else: pass

dense_model = tf.keras.Model(name="model_1",inputs=inputs, outputs=x)
dense_model.summary()


# Define a training loop
epochs = range(300)    
# print(y_train[:5],dense_model(X_train[:5]))
# print(loss(y_train[:5],dense_model(X_train[:5])))
# print(loss(y_train[:5], dense_model(X_train[:5])))
# Do the training
training_loop(dense_model, X_train, y_train,validset=[X_test,y_test])


from sklearn import metrics
y_pred_test = dense_model(X_test).numpy()
y_pred_test = np.argmax(y_pred_test, axis=1).tolist()
# y_test = np.argmax(y_test, axis=1).tolist()

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_test))

# Print the precision and recall, among other metrics
# print(metrics.classification_report(y_test, y_pred_test, digits=3))


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_51 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_329 (Dense)            (None, 8)                 40        
_________________________________________________________________
dense_330 (Dense)            (None, 16)                144       
_________________________________________________________________
dense_331 (Dense)            (None, 32)                544       
_________________________________________________________________
dense_332 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_333 (Dense)            (None, 8)                 136       
_________________________________________________________________
dense_334 (Dense)            (None, 3)                 27  

In [293]:
with tf.GradientTape() as t:
    # Trainable variables are automatically tracked by GradientTape
    current_loss = loss(y_true=y_train,y_pred=dense_model(X_train))

# Use GradientTape to calculate the gradients with respect to W and b
grads= t.gradient(current_loss, dense_model.trainable_variables)
# print(grads)
# print(dense_model.trainable_variables)
# print('ok')
# optimizer.apply_gradients(zip(grads,dense_model.trainable_variables))
# print(optimizer.apply_gradients(zip(grads,dense_model.trainable_variables)))
# print(current_loss)
# print(optimizer.iterations.numpy())

In [301]:
# Define a training loop
epochs = range(300)    
# print(y_train[:5],dense_model(X_train[:5]))
# print(loss(y_train[:5],dense_model(X_train[:5])))
# print(loss(y_train[:5], dense_model(X_train[:5])))
# Do the training
training_loop(dense_model, X_train, y_train,validset=[X_test,y_test])


from sklearn import metrics
y_pred_test = dense_model(X_test).numpy()
y_pred_test = np.argmax(y_pred_test, axis=1).tolist()
# y_test = np.argmax(y_test, axis=1).tolist()

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_test))

# Print the precision and recall, among other metrics
# print(metrics.classification_report(y_test, y_pred_test, digits=3))

Epoch  0: loss=1.08465  val_loss=1.08620
Epoch  1: loss=1.07699  val_loss=1.07819
Epoch  2: loss=1.07047  val_loss=1.07135
Epoch  3: loss=1.06500  val_loss=1.06565
Epoch  4: loss=1.06111  val_loss=1.06165
Epoch  5: loss=1.05775  val_loss=1.05838
Epoch  6: loss=1.05462  val_loss=1.05536
Epoch  7: loss=1.05170  val_loss=1.05220
Epoch  8: loss=1.04872  val_loss=1.04927
Epoch  9: loss=1.04572  val_loss=1.04611
Epoch 10: loss=1.04276  val_loss=1.04318
Epoch 11: loss=1.03986  val_loss=1.04017
Epoch 12: loss=1.03692  val_loss=1.03718
Epoch 13: loss=1.03395  val_loss=1.03412
Epoch 14: loss=1.03093  val_loss=1.03107
Epoch 15: loss=1.02785  val_loss=1.02790
Epoch 16: loss=1.02472  val_loss=1.02469
Epoch 17: loss=1.02156  val_loss=1.02133
Epoch 18: loss=1.01831  val_loss=1.01803
Epoch 19: loss=1.01500  val_loss=1.01448
Epoch 20: loss=1.01155  val_loss=1.01094
Epoch 21: loss=1.00773  val_loss=1.00709
Epoch 22: loss=1.00145  val_loss=1.00061
Epoch 23: loss=0.99573  val_loss=0.99489
Epoch 24: loss=0